In [73]:
import pandas as pd
import numpy as np
import warnings
from unidecode import unidecode

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

warnings.filterwarnings("ignore")

iso_df = pd.read_csv('input/isocodes.csv')  
countries = pd.read_csv('input/countries.csv')
codes_original = pd.read_json('input/geonames.json')

# nan_counts_pc = []
# nan_counts_l = []

# for index, row in iso_df.iterrows():

iso2 = 'GB'
iso3 = 'GBR'

country = countries.loc[countries['alpha-3'] == iso3, 'name'].iloc[0]


In [74]:
# Crunchbase Dataframe
df = pd.read_csv('input/foodtech.csv')
df = df[df['country_code'] == iso3]
df = df[['uuid','country_code', 'state_code', 'region', 'city', 'address', 'postal_code']]

df[['city', 'region']] = df[['city', 'region']].apply(lambda col: col.apply(unidecode).str.lower())
df.rename(columns={'postal_code': 'pc_cb'}, inplace=True)

columns_to_add = {'pc_fill': np.nan, 'lat_fill': np.nan, 'long_fill': np.nan}
df = df.assign(**columns_to_add)


In [75]:
# Postal Codes
codes = codes_original.copy()     
codes = codes[codes['country_code'] == iso2]

codes = codes[['postal_code', 'latitude', 'longitude', 'place_name', 'admin_name1', 'admin_name2','admin_name3',]].copy()

columns_to_normalize = ['admin_name1', 'admin_name2', 'admin_name3', 'place_name']

# Lowercase and apply unidecode for each column
for col in columns_to_normalize:
    codes[col] = codes[col].str.lower()
    codes[col] = codes[col].apply(lambda x: unidecode(x) if x is not None else None)

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10825 entries, 12 to 163812
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   uuid          10825 non-null  object 
 1   country_code  10825 non-null  object 
 2   state_code    0 non-null      object 
 3   region        10825 non-null  object 
 4   city          10825 non-null  object 
 5   address       8500 non-null   object 
 6   pc_cb         8439 non-null   object 
 7   pc_fill       0 non-null      float64
 8   lat_fill      0 non-null      float64
 9   long_fill     0 non-null      float64
dtypes: float64(3), object(7)
memory usage: 930.3+ KB


In [77]:
def exact_match(codes, df, cb_cols, pc_cols, num):

    codes_copy = codes.copy()
    codes_copy.drop_duplicates(subset=pc_cols, inplace=True)

    df1 = pd.merge(df, codes_copy, left_on=cb_cols, right_on=pc_cols, how='left')

    print(df1.info())
    df1['pc_fill'].fillna(df1['postal_code'], inplace=True)
    df1['lat_fill'].fillna(df1['latitude'], inplace=True)
    df1['long_fill'].fillna(df1['longitude'], inplace=True)

    df1 = df1.iloc[:, 0:num]
    
    return df1


In [78]:
# Define the combinations of columns
combos = [
    (['city', 'region'], ['place_name', 'admin_name1']),
    (['city', 'region'], ['admin_name2', 'admin_name1']),
    (['city', 'region'], ['admin_name3', 'admin_name1']),
    (['city'], ['place_name'])
]

# Iterate through each combination
merged_df = df.copy()
for cb_cols, pc_cols in combos:
    merged_df = exact_match(codes, merged_df, cb_cols, pc_cols, 10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10825 entries, 0 to 10824
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   uuid          10825 non-null  object 
 1   country_code  10825 non-null  object 
 2   state_code    0 non-null      object 
 3   region        10825 non-null  object 
 4   city          10825 non-null  object 
 5   address       8500 non-null   object 
 6   pc_cb         8439 non-null   object 
 7   pc_fill       0 non-null      float64
 8   lat_fill      0 non-null      float64
 9   long_fill     0 non-null      float64
 10  postal_code   2743 non-null   object 
 11  latitude      2743 non-null   float64
 12  longitude     2743 non-null   float64
 13  place_name    2743 non-null   object 
 14  admin_name1   2743 non-null   object 
 15  admin_name2   2743 non-null   object 
 16  admin_name3   0 non-null      object 
dtypes: float64(5), object(12)
memory usage: 1.5+ MB
None
<class 'pandas.c

In [79]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10825 entries, 0 to 10824
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   uuid          10825 non-null  object 
 1   country_code  10825 non-null  object 
 2   state_code    0 non-null      object 
 3   region        10825 non-null  object 
 4   city          10825 non-null  object 
 5   address       8500 non-null   object 
 6   pc_cb         8439 non-null   object 
 7   pc_fill       10434 non-null  object 
 8   lat_fill      10434 non-null  float64
 9   long_fill     10434 non-null  float64
dtypes: float64(2), object(8)
memory usage: 930.3+ KB


In [80]:
ref = pd.read_csv('input/asia_cities_mapped.csv')
ref = ref[ref['old_country'] == country]
ref.drop_duplicates(subset=['old_city', 'old_region'], inplace=True)

for col in ref.columns:
    ref[col] = ref[col].str.lower()

if not ref.empty:
    merged_df = pd.merge(merged_df, ref, how='left', left_on=['city'], right_on=['old_city'])

    combos = [
    (['new_city'], ['place_name']),
    (['new_city'], ['admin_name1']),
    (['new_city'], ['admin_name2']),
    (['new_city'], ['admin_name3'])
]

# Iterate through each combination
    for pc_cols, cb_cols in combos:
        merged_df = exact_match(codes, merged_df, pc_cols, cb_cols, 11)

In [81]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10825 entries, 0 to 10824
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   uuid          10825 non-null  object 
 1   country_code  10825 non-null  object 
 2   state_code    0 non-null      object 
 3   region        10825 non-null  object 
 4   city          10825 non-null  object 
 5   address       8500 non-null   object 
 6   pc_cb         8439 non-null   object 
 7   pc_fill       10434 non-null  object 
 8   lat_fill      10434 non-null  float64
 9   long_fill     10434 non-null  float64
dtypes: float64(2), object(8)
memory usage: 930.3+ KB


In [82]:
ref = pd.read_csv('input/world_cities_mapped.csv')
ref = ref[ref['ISO'] == iso3]
ref.drop_duplicates(subset=['old_city'], inplace=True)

for col in ref.columns:
    ref[col] = ref[col].str.lower()

if not ref.empty:
    merged_df = pd.merge(merged_df, ref, how='left', left_on=['city'], right_on=['old_city'])

    combos = [
    (['old_city'], ['place_name']),
    (['old_city'], ['admin_name1']),
    (['old_city'], ['admin_name2']),
    (['old_city'], ['admin_name3'])
]

# Iterate through each combination
    for pc_cols, cb_cols in combos:
        merged_df = exact_match(codes, merged_df, pc_cols, cb_cols, 17)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10825 entries, 0 to 10824
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   uuid          10825 non-null  object 
 1   country_code  10825 non-null  object 
 2   state_code    0 non-null      object 
 3   region        10825 non-null  object 
 4   city          10825 non-null  object 
 5   address       8500 non-null   object 
 6   pc_cb         8439 non-null   object 
 7   pc_fill       10434 non-null  object 
 8   lat_fill      10434 non-null  float64
 9   long_fill     10434 non-null  float64
 10  old_city      115 non-null    object 
 11  country       115 non-null    object 
 12  Coords        115 non-null    object 
 13  ISO           115 non-null    object 
 14  city_mapped   115 non-null    object 
 15  long_mapped   115 non-null    object 
 16  lat_mapped    115 non-null    object 
 17  postal_code   90 non-null     object 
 18  latitude      90 non-null 

In [ ]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8155 entries, 0 to 8154
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   uuid          8155 non-null   object 
 1   country_code  8155 non-null   object 
 2   state_code    0 non-null      object 
 3   region        8155 non-null   object 
 4   city          8155 non-null   object 
 5   address       5425 non-null   object 
 6   pc_cb         5503 non-null   object 
 7   pc_fill       8037 non-null   object 
 8   lat_fill      8037 non-null   float64
 9   long_fill     8037 non-null   float64
 10  new_city      8073 non-null   object 
 11  old_city      169 non-null    object 
 12  country       169 non-null    object 
 13  Coords        169 non-null    object 
 14  ISO           169 non-null    object 
 15  city_mapped   169 non-null    object 
 16  long_mapped   169 non-null    object 
 17  lat_mapped    169 non-null    object 
dtypes: float64(2), object(16)
me

In [ ]:
merged_df['lat_fill'].fillna(merged_df['lat_mapped'], inplace=True)
merged_df['long_fill'].fillna(merged_df['long_mapped'], inplace=True)

merged_df = merged_df.iloc[:, 0:11]

merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8155 entries, 0 to 8154
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   uuid          8155 non-null   object
 1   country_code  8155 non-null   object
 2   state_code    0 non-null      object
 3   region        8155 non-null   object
 4   city          8155 non-null   object
 5   address       5425 non-null   object
 6   pc_cb         5503 non-null   object
 7   pc_fill       8037 non-null   object
 8   lat_fill      8041 non-null   object
 9   long_fill     8041 non-null   object
 10  new_city      8073 non-null   object
dtypes: object(11)
memory usage: 764.5+ KB


In [ ]:
# Define a function for substring matching and selecting the best match
def substring_select_best_match(row, codes, left_col, right_cols):
    best_match = None
    best_postal_code = None
    best_lat = None
    best_long = None
    
    for right_col in right_cols:
        # Filter out NaN values before applying str.contains
        matches = codes[right_col].str.contains(row[left_col], case=False, na=False)
        if any(matches):
            best_match = codes.loc[matches, right_col].iloc[0]
            best_postal_code = codes.loc[matches, 'postal_code'].iloc[0]
            best_lat = codes.loc[matches, 'latitude'].iloc[0]
            best_long = codes.loc[matches, 'longitude'].iloc[0]
            break  # Exit loop after finding a match
    
    return best_match, best_postal_code, 100, best_lat, best_long  # Always return best_score as 100

# Filter rows where pc_fill is NaN
filtered_df = merged_df[merged_df['pc_fill'].isna()]

if not filtered_df.empty:  # Check if filtered_df is not empty

    # Define the columns for substring containment
    left_on = 'city'
    right_on = ['place_name', 'admin_name2', 'admin_name3']

    # Apply substring matching and select best match for each row in filtered_df
    matches_df = filtered_df.apply(lambda row: substring_select_best_match(row, codes, left_on, right_on), axis=1, result_type='expand')
    matches_df.columns = ['best_match', 'best_postal_code', 'best_score', 'best_lat', 'best_long']

    matches_df.replace('', pd.NA, inplace=True)

    merged_df = pd.merge(merged_df, matches_df, how='left', left_index=True, right_index=True)
    merged_df['pc_fill'].fillna(merged_df['best_postal_code'], inplace=True)
    merged_df['lat_fill'].fillna(merged_df['best_lat'], inplace=True)
    merged_df['long_fill'].fillna(merged_df['best_long'], inplace=True)

    merged_df = merged_df.iloc[:, 0:11]
else:
    print("filtered_df is empty. Skipping substring matching and subsequent tasks.")


In [ ]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8155 entries, 0 to 8154
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   uuid          8155 non-null   object
 1   country_code  8155 non-null   object
 2   state_code    0 non-null      object
 3   region        8155 non-null   object
 4   city          8155 non-null   object
 5   address       5425 non-null   object
 6   pc_cb         5503 non-null   object
 7   pc_fill       8108 non-null   object
 8   lat_fill      8109 non-null   object
 9   long_fill     8109 non-null   object
 10  new_city      8073 non-null   object
dtypes: object(11)
memory usage: 1022.6+ KB


In [ ]:
#### 4. EXISTING POSTAL CODES ####

# Across rows: df
codes_df = merged_df[merged_df['pc_cb'].notna()]

codes_df = codes_df[['region', 'city', 'pc_cb']]
codes_df.drop_duplicates(subset=['region', 'city'], inplace=True)

merged_df = pd.merge(merged_df, codes_df, on=['region', 'city'], how='left')
merged_df['pc_fill'].fillna(merged_df['pc_cb_y'], inplace=True)

merged_df = merged_df.iloc[:, 0:10]

In [ ]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8155 entries, 0 to 8154
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   uuid          8155 non-null   object
 1   country_code  8155 non-null   object
 2   state_code    0 non-null      object
 3   region        8155 non-null   object
 4   city          8155 non-null   object
 5   address       5425 non-null   object
 6   pc_cb_x       5503 non-null   object
 7   pc_fill       8149 non-null   object
 8   lat_fill      8109 non-null   object
 9   long_fill     8109 non-null   object
dtypes: object(10)
memory usage: 700.8+ KB


In [ ]:

# # Save to csv
# merged_df = merged_df.rename(columns={'postal_code_x': 'pc_crunchbase',
#                                         'postal_code_y_x': 'pc_filled',
#                                         'latitude_x_x':    'latitude',
#                                         'longitude_x_x':   'longitude'})

# merged_df.to_csv('general/test/' + iso3 + '_processed_test.csv')

# # Checking NaNs
# nan_count_pc = merged_df['pc_filled'].isna().sum()
# nan_counts_pc.append({'iso3': iso3, 'nan_count': nan_count_pc})

# nan_count_l = merged_df['latitude'].isna().sum()
# nan_counts_l.append({'iso3': iso3, 'nan_count': nan_count_l})

# # Create a DataFrame from the list of dictionaries
# nan_counts_df_pc = pd.DataFrame(nan_counts_pc)
# nan_counts_df_l = pd.DataFrame(nan_counts_l)

